In [ ]:
import csv 
import os
from google.cloud import storage

In [ ]:
bucket_name = "fmcdata_nlp"
input_data_prefix = "entity_extraction/" #location of PDF files within the bucket defined by bucket_name
jsonl_prefix = "jsonl/" #path you want the jsonl files to be uploaded to in your bucket
jsonl_output_dir = "/home/jupyter/jsonl/" #temp output directory for jsonl files in your notebook server
csv_output_dir = "/home/jupyter/" #output location of the csv on your notebook server
csv_name = "entity.csv" #csv file name

In [ ]:
if not os.path.exists(jsonl_output_dir):
    os.makedirs(jsonl_output_dir)

In [ ]:
def list_blobs_with_prefix(bucket_name, prefix, delimiter):
    file_list = []
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(
        bucket_name, prefix=prefix, delimiter=delimiter
    )
    for blob in blobs:
        pass
    if delimiter:
        for prefix in blobs.prefixes:
            file_list.append(prefix)
    return file_list

In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

In [ ]:
def write_jsonl(jsonl_string,counter):
    with open(jsonl_output_dir+'input{}.jsonl'.format(counter),'w') as file:
        file.write(jsonl_string)
    upload_blob(bucket_name,jsonl_output_dir+'input{}.jsonl'.format(counter),jsonl_prefix+"input{}.jsonl".format(counter))

In [ ]:
def write_csv(csv_string):
    with open(csv_output_dir+csv_name,'a+') as file:
        file.write(csv_string)

In [ ]:
input_files = list_blobs_with_prefix(bucket_name,input_data_prefix,"pdf")

In [ ]:
counter = 0
for file in input_files:
    json_string = "{{\"document\":{{\"input_config\":{{\"gcs_source\":{{\"input_uris\":[\"gs://{}/{}\"]}}}}}}}}".format(bucket_name,file)
    write_jsonl(json_string,counter)
    counter += 1

In [ ]:
jsonl_files = list_blobs_with_prefix(bucket_name,jsonl_prefix,"jsonl")
for jsonl in jsonl_files:
    csvline = ",gs://{}/{}\n".format(bucket_name,jsonl)
    write_csv(csvline)
upload_blob(bucket_name,csv_output_dir+csv_name,csv_name)